In [1]:
import os
os.chdir('../')

In [2]:
!pwd

/home/snow/Pictures/Kidney-Disease-Classification-Deep-learning-project


In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class PreparedBasedModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: int 
    params_include_top: bool
    params_weights: str 
    params_classess: int

In [4]:
from src.project.constants import *
from src.project.utils.common import create_directory,read_yaml

In [5]:
class configmanager:
    def __init__(
        self,
        config_path = CONFIG_FILE_PATH,
        params_path = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        
        create_directory([self.config.artifacts_root])
    
    
    def BaseModelManager(self) -> PreparedBasedModelConfig:
        config = self.config.prepare_base_model
        params = self.params
        
        create_directory([config.root_dir])
        
        get_base_model = PreparedBasedModelConfig(
            root_dir = Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size= params.IMAGE_SIZE,
            params_learning_rate=params.LEARNING_RATE,
            params_classess= params.CLASSESS,
            params_weights=params.WEIGHTS,
            params_include_top=params.INCLUDE_TOP
        )
        
        return get_base_model

In [6]:
import tensorflow as tf
from zipfile import ZipFile

2024-02-22 10:17:22.669822: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-22 10:17:24.284522: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-22 10:17:26.211738: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-22 10:17:26.211902: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-22 10:17:26.500560: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [13]:
class PrepareBaseModel:
    
    def __init__(self,config:PreparedBasedModelConfig):
        self.config = config
    
    def base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape = self.config.params_image_size,
            weights = self.config.params_weights,
            include_top = self.config.params_include_top
        )
        self.save_model(path = self.config.base_model_path,model=self.model)
        
    @staticmethod
    def save_model(path:Path,model=tf.keras.Model):
        model.save(path)        
            
    @staticmethod
    def _prepare_full_model(model,classess,freeze_all,learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False 
        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
                units = classess,
                activation='softmax'
            )(flatten_in)
            
        full_model = tf.keras.models.Model(
                inputs = model.input,
                outputs = prediction
            )
            
        full_model.compile(
                optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate),
                loss = tf.keras.losses.CategoricalCrossentropy(),
                metrics = ['accuracy']
            )
            
        full_model.summary()
        return full_model
    
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model= self.model,
            classess=self.config.params_classess,
            freeze_all=True,
            learning_rate=self.config.params_learning_rate
        )
        self.save_model(path=self.config.updated_base_model_path,model=self.full_model)
         

In [11]:
from src.project.exception import CustomException
import sys

In [14]:
try:
    config = configmanager()
    config = config.BaseModelManager()
    base_model = PrepareBaseModel(config=config)
    base_model.base_model()
    base_model.update_base_model()
except Exception as e:
    raise CustomException(e,sys)

/home/snow/Pictures/Kidney-Disease-Classification-Deep-learning-project/diesease/lib/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     